In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import accuracy_score


In [2]:
drop_labels = ['Driver', 'DriverNumber', 'LapNumber', 'Compound',
                     'TyreLife', 'Year', 'GrandPrix', 'AirTemp', 'Humidity', 'Pressure',
                     'TrackTemp', 'WindDirection', 'WindSpeed']

In [3]:
df=pd.read_csv('lap_data\\[2023, 2024].csv')
df.columns

Index(['Driver', 'DriverNumber', 'LapTime', 'LapNumber', 'Compound',
       'TyreLife', 'Year', 'GrandPrix', 'AirTemp', 'Humidity', 'Pressure',
       'TrackTemp', 'WindDirection', 'WindSpeed'],
      dtype='object')

In [4]:
for cols in df.columns:
    print(cols+"  \t\t"+str(df[cols].dtype))

Driver  		object
DriverNumber  		int64
LapTime  		float64
LapNumber  		float64
Compound  		object
TyreLife  		float64
Year  		int64
GrandPrix  		object
AirTemp  		float64
Humidity  		float64
Pressure  		float64
TrackTemp  		float64
WindDirection  		int64
WindSpeed  		float64


In [5]:
for cols in df.columns:
    print(cols+'  \t\t'+str(df[cols].isna().sum()))

Driver  		0
DriverNumber  		0
LapTime  		583
LapNumber  		0
Compound  		0
TyreLife  		0
Year  		0
GrandPrix  		0
AirTemp  		0
Humidity  		0
Pressure  		0
TrackTemp  		0
WindDirection  		0
WindSpeed  		0


In [6]:
df.dropna(axis=0,how='any',inplace=True)

In [7]:
for cols in df.columns:
    print(cols+' '+str(df[cols].isna().sum()))

Driver 0
DriverNumber 0
LapTime 0
LapNumber 0
Compound 0
TyreLife 0
Year 0
GrandPrix 0
AirTemp 0
Humidity 0
Pressure 0
TrackTemp 0
WindDirection 0
WindSpeed 0


In [12]:
from sklearn.preprocessing import StandardScaler,OrdinalEncoder
from tensorflow import keras
from keras.utils import to_categorical

scale_cols = ['LapNumber', 'TyreLife', 'AirTemp', 'TrackTemp', 'WindSpeed']
categorical_cols = ['GrandPrix', 'Compound', 'DriverNumber', 'WindDirection']
encoder = OrdinalEncoder()
scaler = StandardScaler()
df['GrandPrix']=encoder.fit_transform(np.array(df['GrandPrix']).reshape(-1,1))
df['Compound']=encoder.fit_transform(np.array(df['Compound']).reshape(-1,1))
df['DriverNumber']=encoder.fit_transform(np.array(df['DriverNumber']).reshape(-1,1))
df['WindDirection']=encoder.fit_transform(np.array(df['WindDirection']).reshape(-1,1))
df[scale_cols]=scaler.fit_transform(df[scale_cols])


In [13]:
df.shape

(43913, 14)

In [14]:
for cols in df.columns:
    print(cols+' '+str(df[cols].isna().sum()))

Driver 0
DriverNumber 0
LapTime 0
LapNumber 0
Compound 0
TyreLife 0
Year 0
GrandPrix 0
AirTemp 0
Humidity 0
Pressure 0
TrackTemp 0
WindDirection 0
WindSpeed 0


In [15]:
x=df.drop(labels=['LapTime','Driver'],axis=1).dropna(axis=0,how='any')
y=df.drop(labels=drop_labels,axis=1).dropna(axis=0,how='any')
x=x.to_numpy()
y=y.to_numpy()

print(x.shape)
print(y.shape)

(43913, 12)
(43913, 1)


In [16]:
from sklearn.linear_model import LinearRegression

rgs = LinearRegression(positive=True,fit_intercept=True)
score = cross_val_score(estimator=rgs,X=x,y=y)
print('\nCross Validation: ',score)
print('\nMean accuracy: ',np.mean(np.array(score)))


Cross Validation:  [-2.23736025e-01  5.03289959e-02 -1.95991502e+00  2.63247189e-04
 -9.24672679e-02]

Mean accuracy:  -0.4451052131734417


In [17]:
from sklearn.ensemble import RandomForestRegressor

rndm = RandomForestRegressor(n_estimators=10)
score = cross_val_score(estimator=rndm,X=x,y=y)
print('\nCross Validation: ',score)
print('\nMean accuracy: ',np.mean(np.array(score)))

c:\Users\Ghost\anaconda3\envs\py310\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\Ghost\anaconda3\envs\py310\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\Ghost\anaconda3\envs\py310\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\Ghost\anaconda3\envs\py310\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Pleas


Cross Validation:  [-8.51216637e+01 -3.55060529e-01 -1.36885411e+00 -4.78475405e-04
 -3.53362247e+00]

Mean accuracy:  -18.075935851411096


In [20]:
# Importing necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
import joblib

# Step 1: Load your dataset
# Assuming 'LapTime' is the target variable in your dataset
# Replace 'data.csv' with your actual data file path
df = pd.read_csv('lap_data\\[2023, 2024].csv')

# Step 2: Data Preprocessing

# 2a. Handle categorical features: OneHotEncoding for ['DriverNumber', 'Compound', 'GrandPrix', 'WindDirection']
categorical_features = ['DriverNumber', 'Compound', 'GrandPrix', 'WindDirection']
encoder = OneHotEncoder(sparse=False, drop='first')  # drop='first' to avoid multicollinearity

encoded_data = encoder.fit_transform(df[categorical_features])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_features))

# 2b. Join the encoded features back to the original dataframe
df = df.join(encoded_df)

# 2c. Drop the original categorical columns
df.drop(columns=categorical_features, inplace=True)

# 2d. Handle missing data (optional): Impute or drop missing values
#df.fillna(df.mean(), inplace=True)

# 2e. Feature Scaling: Standardize numerical columns ['LapNumber', 'TyreLife', 'AirTemp', 'TrackTemp', 'WindSpeed']
numerical_features = ['LapNumber', 'TyreLife', 'AirTemp', 'TrackTemp', 'WindSpeed']

scaler = StandardScaler()
df[numerical_features] = scaler.fit_transform(df[numerical_features])

# Step 3: Model Training
df.dropna(axis=0,how='any',inplace=True)
# 3a. Define features (X) and target (y)
X = df.drop(columns=['LapTime','Driver'])  # Assuming 'LapTime' is the target variable
y = df['LapTime']

# 3b. Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Train a Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=15,random_state=42)
rf_model.fit(X_train, y_train)

# Step 5: Model Evaluation

# 5a. Predict on the test set
y_pred = rf_model.predict(X_test)

# 5b. Calculate evaluation metrics: MSE and R-squared
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')


c:\Users\Ghost\anaconda3\envs\py310\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Mean Squared Error: 8.428418563141925
R-squared: 0.996045322341599


In [21]:

# Step 6: Hyperparameter Tuning with GridSearchCV

param_grid = {
    'n_estimators': [20, 30, 50],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Step 7: Retrieve the best model
best_rf = grid_search.best_estimator_

# Step 8: Evaluate the tuned model
y_pred_best = best_rf.predict(X_test)
mse_best = mean_squared_error(y_test, y_pred_best)
r2_best = r2_score(y_test, y_pred_best)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

print(f'Tuned Model Mean Squared Error: {mse_best}')
print(f'Tuned Model R-squared: {r2_best}')

# Step 9: Feature Importance (Optional)
importances = best_rf.feature_importances_
features = X_train.columns
feature_importance = pd.DataFrame({'Feature': features, 'Importance': importances})
print(feature_importance.sort_values(by='Importance', ascending=False))

# Step 10: Save the trained model
joblib.dump(best_rf, 'f1_lap_time_predictor.pkl')
